In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import pyautogui as auto
from yaml import safe_load
import time


def show(img) -> None:
    cv2.namedWindow("Screen")
    cv2.imshow("Screen", cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB))
    cv2.waitKey(0)
    cv2.destroyAllWindows()


with open('.//config//runConfig.yaml', 'r') as f:
        config = safe_load(f)
model = YOLO(config['model'], task='detect')
screenX, screenY = auto.size()

while True:
    # detect the blank puzzle
    screen = cv2.cvtColor(np.array(auto.screenshot()), cv2.COLOR_RGB2BGR)
    result = model(
        screen,
        task="detect",
        device=config["device"],
        conf=config["conf"],
        max_det=config["max_det"],
        imgsz=(screenX, screenY),
        iou=config["NMS"],
        stream=False,
    )[0]
    # display(result.boxes)
        
    if result.boxes.cls.numel() > 0:
        arrowXywh = result.boxes.xywh[result.boxes.cls == config['classes']['arrow']].cpu().numpy().astype(int)[0]
        puzzleXywh = result.boxes.xywh[result.boxes.cls == config['classes']['puzzle']].cpu().numpy().astype(int)[0]
        display(arrowXywh)
        # Draw bounding boxes on the screen image
    for box in result.boxes.xyxy:
        x1, y1, x2, y2 = box
        cv2.rectangle(screen, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    show(screen)
    time.sleep(2)


WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
0: 640x1088 (no detections), 122.3ms
Speed: 4.0ms preprocess, 122.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 1088)

WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
0: 640x1088 1 puzzle, 1 arrow, 100.5ms
Speed: 6.6ms preprocess, 100.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 1088)


array([562, 660,  72,  68])

array([833, 540, 114, 108])

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
datasetYaml = ".//config//puzzleBlockDatasets.yaml"
def test():
    # Run batched inference on a list of images
    results = model.val(data=datasetYaml,device='cpu',conf=0.8,max_det=1)

In [ ]:
# Load a model
model = YOLO(".//models//puzzle_s.onnx",task='detect')  # pretrained YOLO11n model

%timeit test()

In [ ]:
# Load a model
model = YOLO(".//models//puzzle_s_openvino_model",task='detect')  # pretrained YOLO11n model

%timeit test()

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO(".//models//puzzle_s.pt",task='detect')  # pretrained YOLO11n model

%timeit test()